In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
%run tinyProp.py    # import all classes

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
# TESTLINEAR LAYER

tpParams = TinyPropParams(1, 1, 1, 0)   # Effectively no TinyProp

device = 'cpu'
torchLin = torch.nn.Linear(10, 6, bias=True)
tinyPropLin = TinyPropLinear(10, 6, tpParams, 1, bias=True)
x = torch.randn(1, 10)
y = x.detach().clone()
x.requires_grad = True
y.requires_grad = True

# Make sure weight and bias are the same
new_weights = torch.randn_like(torchLin.weight)
torchLin.weight = nn.Parameter(new_weights)
tinyPropLin.weight = nn.Parameter(new_weights)

new_bias = torch.randn_like(torchLin.bias)
torchLin.bias = nn.Parameter(new_bias)
tinyPropLin.bias = nn.Parameter(new_bias)

# and check that forwards produces the same result
forward = torchLin(x)
print(">LINEAR forwards:")
#print(forward)
print(">LINEAR backwards:")
forward.backward(torch.ones_like(forward))
#print(torchLin.weight.grad)
#print(torchLin.bias.grad)
print(x.grad)

forward = tinyPropLin(y)
print(">TINYPROP forwards:")
#print(forward)
print(">TINYPROP backwards:")
forward.backward(torch.ones_like(forward))
#print(tinyPropLin.weight.grad)
#print(tinyPropLin.bias.grad)
print(y.grad)

>LINEAR forwards:
>LINEAR backwards:
tensor([[-3.6680,  2.8911,  0.1020, -2.3948, -2.0706,  3.0194, -2.2185, -1.3537,
         -1.0371, -3.6269]])
>TINYPROP forwards:
>TINYPROP backwards:
tensor([[-3.6680,  2.8911,  0.1020, -2.3948, -2.0706,  3.0194, -2.2185, -1.3537,
         -1.0371, -3.6269]])


In [25]:
%run tinyProp.py    # import all classes

In [10]:
# TEST CONVOLUTIONAL LAYER

tpParams = TinyPropParams(1, 1, 1, 0)   # Effectively no TinyProp

torchConv = torch.nn.Conv2d(3, 1, 3)
tinyPropConv = TinyPropConv2d(3, 1, 3, tpParams, 1)
x = torch.randn(1, 3, 12, 12)
y = x.detach().clone()
x.requires_grad = True
y.requires_grad = True

new_weights = torch.randn_like(torchConv.weight)
torchConv.weight = nn.Parameter(new_weights)
tinyPropConv.weight = nn.Parameter(new_weights)

new_bias = torch.randn_like(torchConv.bias)
torchConv.bias = nn.Parameter(new_bias)
tinyPropConv.bias = nn.Parameter(new_bias)

# and check that forwards produces the same result
forward = torchConv(x)
print(">CONV forwards:")
#print(forward)
print(">LINEAR backwards:")
forward.backward(torch.ones_like(forward))
print(torchConv.weight.grad)
print(torchConv.bias.grad)
#print(x.grad)

forward = tinyPropConv(y)
print(">TINYPROP forwards:")
#print(forward)
print(">TINYPROP backwards:")
forward.backward(torch.ones_like(forward))
print(tinyPropConv.weight.grad)
print(tinyPropConv.bias.grad)
#print(y.grad)

>CONV forwards:
>LINEAR backwards:
tensor([[[[ 0.3828, -0.3665, -9.6231],
          [-7.2493, -5.0602, -9.5640],
          [-1.7017, -0.6819, -5.3610]],

         [[ 7.9566,  5.1179,  7.0664],
          [ 7.9504,  6.3732,  8.8559],
          [15.8044, 16.9372, 20.0440]],

         [[ 4.0545,  7.0025,  9.7788],
          [ 3.9602,  6.7449, 10.9203],
          [-2.2036,  3.5207,  6.2683]]]])
tensor([100.])
>TINYPROP forwards:
>TINYPROP backwards:
tensor([[[[ 0.3828, -0.3665, -9.6231],
          [-7.2493, -5.0602, -9.5640],
          [-1.7017, -0.6818, -5.3610]],

         [[ 7.9566,  5.1179,  7.0664],
          [ 7.9504,  6.3732,  8.8559],
          [15.8044, 16.9372, 20.0440]],

         [[ 4.0545,  7.0025,  9.7788],
          [ 3.9602,  6.7449, 10.9203],
          [-2.2036,  3.5207,  6.2683]]]])
tensor([100.])
